In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from tqdm.notebook import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

## Convolutional Neural Network
<br></br>
$Conv\ shape = \frac{input\ size - kernel\ size + padding * 2}{stride}+1$
<br></br>
<br></br>
실제 구현은 Convolution 연산이 아닌 Cross Correlation 연산으로 구현됨

필터 방향 차이에 의해 출력 텐서의 방향은 달라져도 value는 같기에

뒤집을 필요가 없는 Cross Correlation 사용

## Max Pooling
<br></br>
해당 커널에서 최대값만 추출
<br></br>
<br></br>
<br></br>
Conv, MaxP 모두 Overfitting을 방지하는 효과

In [6]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape = (batch_size, 1, 28, 28)
        #    Conv  shape = (batch_size, 32, 28, 28)
        #    MaxPl shape = (batch_size, 32, 14, 14)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, 3, stride=1, padding=1), # (batch_size, 64, 14, 14)
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))     # (batch_size, 64, 7, 7)
        
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # Flatten
        out = self.fc(out)
        
        return out

In [7]:
model = CNN().to(device)

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in tqdm(range(training_epochs)):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device) # Labeled numbers, not one-hot encoded
        
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    
print('Learning Finished!')

Learning started. It takes sometime.


[Epoch:    1] cost = 0.22401993
[Epoch:    2] cost = 0.0621708035
[Epoch:    3] cost = 0.0448966809
[Epoch:    4] cost = 0.0354979858
[Epoch:    5] cost = 0.0290464852
[Epoch:    6] cost = 0.0249115285
[Epoch:    7] cost = 0.0205448903
[Epoch:    8] cost = 0.0182830133
[Epoch:    9] cost = 0.0149991205
[Epoch:   10] cost = 0.0126367733
[Epoch:   11] cost = 0.0106618488
[Epoch:   12] cost = 0.00994431414
[Epoch:   13] cost = 0.00813665893
[Epoch:   14] cost = 0.00735676754
[Epoch:   15] cost = 0.00582294166

Learning Finished!


In [10]:
with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9840999841690063
